In [9]:
#default_exp train_2

In [9]:
#export
import sys
# import libraries
import pandas as pd
import sqlalchemy as db
import nltk
nltk.download(['punkt', 'wordnet'])

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('omw-1.4')
import numpy as np

import warnings
warnings.filterwarnings('ignore')
import pickle


from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sanje\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sanje\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sanje\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sanje\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
#export
def load_data(database_filepath):
    engine = db.create_engine('sqlite:///'+database_filepath)
    df=pd.read_sql_query("SELECT * FROM tweets;", engine)
    X = df['message']
    y = df.drop(['id','categories','message','original','genre'], axis=1)
    y_cols= y.columns.tolist()
    return X,y,y_cols

In [11]:
engine = db.create_engine('sqlite:///'+'../data/DisasterResponse.db')
df=pd.read_sql_query("SELECT * FROM tweets;", engine)
X = df['message']
y = df.drop(['id','categories','message','original','genre'], axis=1)
y_cols= y.columns.tolist()
df.head()

,id,message,original,genre,categories,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#export
def tokenize(example_sent):
    example_sent = example_sent.lower()
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(example_sent)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    wordnet_lemmatizer = WordNetLemmatizer()
    filtered_Lemm_sentence = [wordnet_lemmatizer.lemmatize(w).strip() for w in filtered_sentence]
    return filtered_Lemm_sentence

In [4]:
#export
def build_model():
    RndmFrst_pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(),n_jobs=20))
    ])    
    return RndmFrst_pipeline
       


def evaluate_model(model, X_test, y_test, category_names):
    prediction = model.predict(X_test)
    for i, cat in enumerate(category_names):
        print(i, cat)
        print('------')
        print(classification_report(y_true= y_test[cat].values.reshape(-1,1), y_pred=prediction[:,i].reshape(-1,1)))
        print('------')
        pass

In [5]:
#export
def save_model(model, model_filepath):
    filehandler = open(model_filepath,"wb")
    pickle.dump(model,filehandler)

In [13]:
#export
def main():
    try:
        if len(sys.argv) == 3:
            database_filepath, model_filepath = sys.argv[1:]
            print('Loading data...\n    DATABASE: {}'.format(database_filepath))
            X, Y, category_names = load_data(database_filepath)
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

            print('Building model...')
            model = build_model()

            print('Training model...')
            model.fit(X_train, Y_train)

            print('Evaluating model...')
            evaluate_model(model, X_test, Y_test, category_names)

            print('Saving model...\n    MODEL: {}'.format(model_filepath))
            save_model(model, model_filepath)

            print('Trained model saved!')

        else:
            print('Please provide the filepath of the disaster messages database '\
                  'as the first argument and the filepath of the pickle file to '\
                  'save the model to as the second argument. \n\nExample: python '\
                  'train_classifier.py ../data/DisasterResponse.db classifier.pkl')
    except:
        pass

In [14]:
#export
if __name__ == '__main__':
    main()

Loading data...
    DATABASE: -f
